# Inverse filter demo used in lecture
This notebook is an example of what happens when an inverse filter is used to identify the presence of a signal.

### Premable
Start by importing the Python libraries that we require

In [ ]:
import numpy as np
import scipy.signal as sps
from IPython import display
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except:
        return False

### User specified parameters
The following parameters can be specified.

Parameter | Meaning
--------- | -------
<code>number_of_samples</code>| length of the time record to evaluate
<code>X1</code> | an example waveform that we wish to detect in a data signal
<code>X2</code> | a second example waveform that we wish to detect

In [ ]:
number_of_samples = 60
X1 = [1, 0.4, 0.14, -0.38, 0.2125]
X2 = [1, -0.7, -0.3, -0.534, 0.6305, -0.23375]

### Plotting Functions

In [ ]:
def plot_stem(x, y, name=''):
    """
       Create a stem plot of inverse filter output.
       
       INPUT:
            x (array-like): The x-positions of the stems. 
            y (array-like): The y-values of the stem heads.
            name  (string): Name of the plot to save
    """
    plt.figure(figsize = (12, 6))
    plt.rcParams.update({'font.size': 16})
    
    (markerLines, stemLines, baseLines) = plt.stem(x, y, use_line_collection = True)
    plt.setp(baseLines, color = 'black', linewidth=1) 
    markerLines.set_markerfacecolor('none')
    
    plt.xlabel('Sample')
    plt.xlim([0, x[-1]])
    plt.ylabel('Output')
    plt.title('Inverse filter output')
    
    plt.show()
    
    if not is_jupyter() and not name:
        plt.tight_layout()
        plt.savefig(name)

In [ ]:
def pz_plot(z, p):
    """
       Plot the poles and zeros.
       
       INPUT:
           z (complex number): Represent zeros.
           p (complex number): Represent poles.
           name  (string): Name of plot file to produce.
    """
    
    # Create a plot figure
    plt.figure(figsize = (12, 6))
    plt.rcParams.update({'font.size': 16})
    plt.ion()
    ax = plt.gca()
    
    # Set two axes with same standard tick length
    ax.set_aspect(1)
    
    # Plot the unit circle 
    circle1 = plt.Circle((0, 0), 1, 
                         color = '#87cefa', 
                         fill = False,
                         linestyle = 'dotted')
    
    # Plot the two straight lines perpendicular to 
    # each other through the origin
    ax.plot(np.linspace(-2, 2, 100), 
            np.zeros(100), 
            color = '#87cefa', 
            linestyle = 'dotted')
    ax.plot(np.zeros(100),
            np.linspace(-1, 1, 100),  
            color = '#87cefa',
            linestyle = 'dotted')
    
    # Plot zeros 
    for i in range(len(z)):
        ax.plot(np.real(z[i]),
                np.imag(z[i]), 
                'bo', 
                markerfacecolor = 'none')
                
    # Plot poles 
    for i in range(len(p)):
        ax.plot(np.real(p[i]), 
                np.imag(p[i]), 
                'bx')
    
    # Plot the origin
    ax.plot(0, 0, 'bo', markerfacecolor = 'none')

    # Tidy up the plot to control axes sizes and labels
    ax.set_xticks(np.linspace(-2, 2, 5))
    ax.set_yticks(np.linspace(-1, 1, 5))
    ax.set_xlabel('Real Part')
    ax.set_ylabel('Imaginary Part')
    ax.set_title('Pole-Zero Plot')
    ax.add_artist(circle1)

### Generate signal
Here we produce a signal that is a noisy input, with the signal that we are looking for at a delay of 30 samples.

In [ ]:
n = np.arange(0, number_of_samples)

noise = np.random.randn(number_of_samples)*0.1
Y = noise.copy()
Y[30:30+len(X1)] = Y[30:30+len(X1)] + X1

### Apply inverse filter
We generate a filter that is given by $H(z)=\frac{1}{X(z)}$, and apply it to the noisy signal, $y(n)$.  From the output, we can see that the signal has been found at the desired location, evidenced by the large value in the inverse filter output.

In [ ]:
# Now use an inverse filter on Y
W = sps.lfilter([1], X1, Y)

# and plot it
plot_stem(n, W)

### Apply the technique again
We repeat the above experiment for the signal $x2(n)$, but this time the output is very different.  Instead of only detecting the presence of the signal within $y(n)$, the magnitude of the inverse filter output grows rapidly, and would continue to grow as we apply the filter to a longer record of the noisy signal.

In [ ]:
# Generate the new noisy signal
Y = noise.copy()
Y[30:30+len(X2)] = Y[30:30+len(X2)] + X2

# Now use an inverse filter on Y
W = sps.lfilter([1], X2, Y)
    
# and plot it
plot_stem(n, W, name='Inverse_filter.pdf')

What has happend is that the inverse filter is unstable, and due to the noise in the input signal, the filter instability expresses itself as exponential growth of the output.  Re-running this script will produce different graphs, but they will all exhibit exponential growth.

### z-transform of signals
Here we plot the z-transforms of the two inverse filters corresponding to the two signals, $x1(n)$, and $x2(n)$.  In the first case, the signal has four zeros, all within the unit circle, so the pole-zero diagram has four poles, all within the unit circle.

In [ ]:
# Find the pole, zeros of response
(z,p,k) = sps.tf2zpk(1,X1)
pz_plot(z, p)

In the second case, the input signal has five zeros, one of which is outside the unit circle, so the inverse filter has five poles.  It is the pole outside the unit circle that produces the instability leading to exponential growth of the output.

In [ ]:
(z1,p1,k1) = sps.tf2zpk(1,X2)
pz_plot(z1, p1)

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.